In [1]:
import re
import json
from timeout_decorator import timeout

In [2]:
file_name = "RegexEval_v3"
out_file = "RegexEval_v4"

In [3]:
with open(f"./{file_name}.json") as f:
    data = json.load(f)

len(data)

1470

In [4]:
with open("done.txt", "r") as f:
    done = f.read().split("\n")

len(done)

1470

In [5]:
@timeout(60)
def match_expression(expression, text):
    try:
        return re.search(expression, text) is not None
    except Exception as e:
        print(e)
        return None

In [7]:
new_data = []
problematic_data = []
for i in range(len(data)):
    item = data[i]


    if "FALSE" in done[i]:
        continue
    
    # if (
    #     item["expression"].strip() == ""
    #     or item["raw_prompt"].strip() == ""
    #     or item["refined_prompt"].strip() == ""
    # ):
    #     continue

    is_valid = True
    item["serial"] = i + 1
    expression = item["expression"].strip()
    raw_prompt = item["raw_prompt"].strip()
    refined_prompt = item["refined_prompt"].strip()
    matches = item["matches"]
    non_matches = item["non_matches"]
    item["problematic"] = []
    
    # print(len(matches), len(non_matches))

    for match in matches:
        try:
            is_match = match_expression(expression, match)
        except TimeoutError:
            print("TimeoutError: ", expression, match)
            is_match = None
        if is_match is None or is_match == False:
            is_valid = False
            item["problematic"].append({"example": match, "is_match": True})
            # break
    for non_match in non_matches:
        try:
            is_match = match_expression(expression, non_match)
        except TimeoutError:
            print("TimeoutError: ", expression, match)
            is_match = None
        if is_match is None or is_match == True:
            is_valid = False
            item["problematic"].append({"example": non_match, "is_match": False})

    if is_valid:
        new_data.append(item)
    else:
        print("Invalid: ", i, expression, raw_prompt, matches, non_matches)
        problematic_data.append(item)


TRUE ^\d$
TRUE ^\d{5}$
TRUE ^\d{5}-\d{4}$
TRUE ^\d{5}$|^\d{5}-\d{4}$
TRUE ^\d{3}-\d{2}-\d{4}$
TRUE ^[a-zA-Z]$
TRUE ^[a-zA-Z]+$
TRUE ^[a-zA-Z0-9]+$
TRUE ^\d+$
TRUE ^(\+|-)?\d+$
TRUE ^[a-zA-Z]\w{3,14}$
TRUE ^\w+@[a-zA-Z_]+?\.[a-zA-Z]{2,3}$
TRUE ^\d{1,2}\/\d{1,2}\/\d{4}$
TRUE ^[1-5]$
TRUE ^[12345]$
TRUE ^[2-9]\d{2}-\d{3}-\d{4}$
TRUE ^\d{5}-\d{4}|\d{5}|[A-Z]\d[A-Z] \d[A-Z]\d$
Invalid:  16 ^\d{5}-\d{4}|\d{5}|[A-Z]\d[A-Z] \d[A-Z]\d$ This expression matches three different formats of postal codes: 5 digit US ZIP code, 5 digit US ZIP code + 4, and 6 digit alphanumeric Canadian Postal Code. The first one must be 5 numeric digits. The ZIP+4 must be 5 numeric digits, a hyphen, and then 4 numeric digits. The Canadian postal code must be of the form ANA NAN where A is any uppercase alphabetic character and N is a numeric digit from 0 to 9. ['44240', '44240-5555', 'G3H 6A3', '40243', '58940', '32854-1383', '73948-1937', '48527-4827', '49284-3194', '48365', 'F7S 9S9', 'S7S 7S7', 'K9K 4J4'] ['Ohio', '

In [8]:
len(new_data)

762

In [9]:
len(problematic_data)

239

In [10]:
with open(f"./{out_file}.json", 'w') as f:
    json.dump(new_data, f)

In [11]:
with open(f"./{out_file}_problematic.json", 'w') as f:
    json.dump(problematic_data, f)

In [12]:
all_data = new_data + problematic_data
with open(f"./{out_file}_all.json", 'w') as f:
    json.dump(all_data, f)

In [13]:
expressions = []
with open('./RegexEval_v4.json') as f:
    data = json.load(f)
    for item in data:
        expressions.append(item["expression"])
len(expressions)

with open('./regexlist.txt', 'w') as f:
    for expression in expressions:
        f.write(f"{expression}\n")
